# Similarity and Distance

피처 사이의 관계는 벡터 사이의 관계로 환원될 수 있다. 벡터 사이의 관계는 다시 서로 얼마나 비슷한가, 반대로 얼마나 떨어져 있는가라는 물음으로 환원될 수 있다. 

따라서 벡터 사이의 유사도()와 거리()를 구해야 할 필요가 생겨난다. 

유사도와 거리는 반대 대체로 반대 개념이지만, 각각의 특징이 있다. 

* 유사도
  - "얼마나 비슷한가"에 대한 값이다.
  - 주로 0에서 1 사이의 값을 가지며, 0은 전혀 같지 않은 상태(벡터에서의 직교와 같은 위상), 1은 완전히 같은 상태.
  
* 거리
  - "얼마나 떨어져 있는가"에 대한 값이다.
  - 주로 0에서 무한대의 값을 가진다. 0은 완전히 같은 상태.


유사도를 구하는 방법은 매우 다양하다. 그 가운데 가장 보편적으로 사용하는 방법은 코사이 유사도(cosine similarity)이다. 

코사인 유사도는 두 벡터가 이루는 내각의 코사인 값을 의미한다. 따라서 직교하면 0이 되고, 완전히 일치하면 1이 된다. 

코사인 유사도는 두 벡터의 내적을 이용하여 간단하게 구할 수 있기 때문에 계산에 있어서도 유리하다. 

거리를 구하는 방법 역시 다양하다. 그 가운데 가장 보편적으로 사용하는 방법은 유클리디안 거리(Euclidian distance)이다. 

유클리디안 거리는 두 벡터 사이의 직선 거리를 의미한다. 따라서 완전히 같은 곳에 있으면 0이 되고, 거리가 늘어남에 따라 크기가 커진다. 


In [ ]:
from collections import Counter
import numpy as np

In [ ]:
corpus_path = "../data/formulas.txt"
corpus_ = open( corpus_path, 'r', encoding='utf-8' ).read()
header, corpus_raw = corpus_.split("***")
corpus_raw = corpus_raw.strip()
corpus = [ line.strip() for line in corpus_raw.split("\n") ]
corpus_tokenized = [ line.split() for line in corpus ]

print( "# Corpus Description" )
print( header.strip() )
print()
print( "# Corpus Size: ", len(corpus) )

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from time import time

In [ ]:
min_df = 6

In [ ]:
# Build TF Matrix and TF-IDF Matrix

t0 = time()
tf_vectorizer = CountVectorizer( min_df=min_df )
tf = tf_vectorizer.fit_transform( corpus )
tf_feature_names = tf_vectorizer.get_feature_names()
print( "TF Matrix done in {:03f}s.".format(time() - t0) )

t0 = time()
tfidf_vectorizer = TfidfVectorizer( min_df=min_df )
tfidf = tfidf_vectorizer.fit_transform( corpus )
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print( "TF-IDF Matrix done in {:03f}s.".format(time() - t0) )

doc_size, feature_size = tf.shape
print( "# 처방 개수: ", doc_size)
print( "# 본초 개수: ", feature_size, " ({}회 이상 사용된 본초)".format( min_df ) ) 

term_vector_via_tf = tf.T
term_vector_via_tfidf = tfidf.T

In [ ]:
co_word = ( tf.T * tf )
term_vector_via_coword = co_word
print( "{}개 본초에 대해 {} 길이의 벡터 생성".format( *co_word.shape ) )

In [ ]:
# Observed Value
observed_v = co_word.toarray()

# Expected Value
margin_x = np.full( co_word.shape, freq_per_term )
margin_y = margin_x.T
expected_v = np.divide(  np.multiply( margin_x, margin_y ),  sum( freq_per_term )   )

# T-score with addone smoothing
observed_v_add1 = np.add( observed_v, 1 )
t_score = np.divide( np.subtract( observed_v, expected_v ) , np.sqrt( observed_v_add1 ) )

from scipy import sparse
term_vector_via_tscore = sparse.csc_matrix( t_score )
print( "{}개 본초에 대해 {} 길이의 벡터 생성".format( *term_vector_via_tscore.shape ) )

In [ ]:
import gensim

vec_size = 64
pochs = 32
max_formula_length = max( size_per_doc ) 
print( "Window size: ", max_formula_length )

w2v = gensim.models.Word2Vec( size=vec_size, window=max_formula_length, min_count=min_df, workers=10 )
w2v_feature_names = w2v.build_vocab( corpus_tokenized )
w2v.train( corpus_tokenized, total_examples=len( corpus_tokenized ), epochs=pochs)


In [ ]:
def w2v_most_similar( test_herb, n=4 ):
    return w2v.wv.most_similar( [ test_herb ], topn=n )

w2v_most_similar( "감초" )

### 3. 결과 비교


#### Neighbors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

class MostElem:
    
    def __init__(self, feature_list, embedding_matrix, max_n=4):
        self.feature_list = feature_list
        self.embedding_matrix = embedding_matrix
        self.max_n = max_n
    
    def most_similar( self, term ):
        i = self.feature_list.index( term )
        sims = cosine_similarity( self.embedding_matrix[i,:], self.embedding_matrix )
        most_sim_idx = sims.argsort()[0][-self.max_n-1:-1].tolist()
        most_sim_idx.reverse()
        return [ ( self.feature_list[i], sims[0][i] ) for i in most_sim_idx ] 


In [ ]:
by_tf = MostElem( tf_feature_names, term_vector_via_tf, max_n=4  )
by_tfidf = MostElem( tfidf_feature_names, term_vector_via_tfidf, max_n=4  )
by_coword = MostElem( tf_feature_names, term_vector_via_coword, max_n=4  )
by_tscore = MostElem( tf_feature_names, term_vector_via_tscore, max_n=4  )

In [ ]:
test_herb = "마황"

print( by_tf.most_similar( test_herb ) )
print( by_tfidf.most_similar( test_herb ) )
print( by_coword.most_similar( test_herb ) )
print( by_tscore.most_similar( test_herb ) )
print( w2v_most_similar( test_herb ) )

In [ ]:
def fm_chain( init_herb, most_fn ):
    output = [init_herb]
    next_herb = init_herb
    i = 0
    while i < 20:
        next_herb = most_fn( next_herb )[0][0]
        if next_herb in output:
            break
        output.append( next_herb )
        i = i + 1

    return output

In [ ]:
print(  fm_chain( "인삼", glove.most_similar ) )